In [38]:
# import the necessary packages
import mysql.connector
from mysql.connector import Error
import pandas as pd
import os
import time

In [39]:
# 連接資料庫
connection = mysql.connector.connect(
    host='localhost',  # 主機名稱
    database='example',  # 資料庫名稱
    user='root',  # 帳號
    password='root'
)  # 密碼
if connection.is_connected():
    # 顯示資料庫版本
    db_Info = connection.get_server_info()
    print("資料庫版本：", db_Info)

    # 顯示目前使用的資料庫
    cur = connection.cursor()
    cur.execute("SELECT DATABASE();")
    record = cur.fetchone()
    print("目前使用的資料庫：", record)

資料庫版本： 5.5.5-10.11.2-MariaDB
目前使用的資料庫： ('example',)


In [40]:
#讀取資料
Data= os.path.join('fake_data.csv') #原始資料
df = pd.read_csv(Data)
df.head()

,id,name,gender,age,height_cm,weight_kg,country,position
0,1,Amy,0,18,160,48,Taiwan,Student
1,2,John,1,25,175,75,United States,Programmer
2,3,Sarah,0,33,168,62,Canada,Doctor
3,4,David,1,42,180,85,Australia,Engineer
4,5,Emily,0,29,162,52,United Kingdom,Teacher


In [41]:
df.dtypes

id            int64
name         object
gender        int64
age           int64
height_cm     int64
weight_kg     int64
country      object
position     object
dtype: object

In [34]:
df.columns

Index(['id', 'name', 'gender', 'age', 'height_cm', 'weight_kg', 'country',
       'position'],
      dtype='object')

In [66]:
#transform datatype (in a simple way)
#for col in df.select_dtypes(include=['int64']):
#    df[col] = df[col].astype('int32')

In [42]:
#transform datatype
df = df.astype('str')

## 以下兩種範例，擇一即可


In [36]:
#Insert
t0 = time.time()
columns = ','.join([f'{x}' for x in df.columns])
param_placeholders = ','.join(['%s' for x in range(len(df.columns))])

def insert(c0, c1, c2, c3, c4, c5, c6, c7):
    try:
        insertStatement = f"INSERT INTO profile ({columns}) VALUES ({param_placeholders})"
        data = (c0, c1, c2, c3, c4, c5, c6, c7)
        cur.execute(insertStatement, data)
        connection.commit()
        print(time.time()-t0)
        print("Successfully added entry to database")
    except Error as e:
        print(f"Error adding entry to database: {e}")

col_m = df.columns.to_list()
for i in range(len(df)):
    insert(df[col_m[0]][i], df[col_m[1]][i], df[col_m[2]][i], df[col_m[3]][i], df[col_m[4]][i], df[col_m[5]][i], df[col_m[6]][i], df[col_m[7]][i])

0.0020008087158203125
Successfully added entry to database
0.003000974655151367
Successfully added entry to database
0.004001617431640625
Successfully added entry to database
0.0050013065338134766
Successfully added entry to database
0.006000041961669922
Successfully added entry to database
0.00700068473815918
Successfully added entry to database
0.008002042770385742
Successfully added entry to database
0.009000062942504883
Successfully added entry to database
0.010000228881835938
Successfully added entry to database
0.012001514434814453
Successfully added entry to database
0.01399993896484375
Successfully added entry to database
0.015001535415649414
Successfully added entry to database
0.016002178192138672
Successfully added entry to database
0.016002178192138672
Successfully added entry to database
0.017000436782836914
Successfully added entry to database
0.018000364303588867
Successfully added entry to database
0.018999338150024414
Successfully added entry to database
0.018999338150

In [43]:
# Insert (精簡版)
t0 = time.time()
columns = ','.join([f'{x}' for x in df.columns])
param_placeholders = ','.join(['%s' for x in range(len(df.columns))])

def insert(*args):
    try:
        insertStatement = f"INSERT INTO profile ({columns}) VALUES ({param_placeholders})"
        cur.execute(insertStatement, args)
        connection.commit()
        print(time.time()-t0)
        print("Successfully added entry to database")
    except Error as e:
        print(f"Error adding entry to database: {e}")

for i in range(len(df)):
    insert(*df.iloc[i])

Error adding entry to database: 1062 (23000): Duplicate entry '1' for key 'PRIMARY'
Error adding entry to database: 1062 (23000): Duplicate entry '2' for key 'PRIMARY'
Error adding entry to database: 1062 (23000): Duplicate entry '3' for key 'PRIMARY'
Error adding entry to database: 1062 (23000): Duplicate entry '4' for key 'PRIMARY'
Error adding entry to database: 1062 (23000): Duplicate entry '5' for key 'PRIMARY'
Error adding entry to database: 1062 (23000): Duplicate entry '6' for key 'PRIMARY'
Error adding entry to database: 1062 (23000): Duplicate entry '7' for key 'PRIMARY'
Error adding entry to database: 1062 (23000): Duplicate entry '8' for key 'PRIMARY'
Error adding entry to database: 1062 (23000): Duplicate entry '9' for key 'PRIMARY'
Error adding entry to database: 1062 (23000): Duplicate entry '10' for key 'PRIMARY'
Error adding entry to database: 1062 (23000): Duplicate entry '11' for key 'PRIMARY'
Error adding entry to database: 1062 (23000): Duplicate entry '12' for key

In [28]:
# Update
columns = df.columns.tolist()
param_placeholders = ','.join([f'{col}=%s' for col in columns if col != 'id'])


## 將id作為primary key以更新其餘欄位 (注意最後的%s為id，因此所對應c7的部分也應為id欄位)
def update(c0, c1, c2, c3, c4, c5, c6, c7):
    try:
        updateStatement = f"UPDATE profile SET {param_placeholders} WHERE id = %s"
        data = (c0, c1, c2, c3, c4, c5, c6, c7)
        cur.execute(updateStatement, data)
        connection.commit()
        print(time.time()-t0)
        print("Successfully updated entry to database")
    except Error as e:
        print(f"Error updating entry to database: {e}")
        
t0 = time.time()
for index, row in df.iterrows():
    update(row['name'], row['gender'], row['age'], row['height_cm'], row['weight_kg'], row['country'], row['position'], row['id'])

0.0009996891021728516
Successfully updated entry to database
0.0019989013671875
Successfully updated entry to database
0.0030007362365722656
Successfully updated entry to database
0.0030007362365722656
Successfully updated entry to database
0.004000186920166016
Successfully updated entry to database
0.004000186920166016
Successfully updated entry to database
0.005001544952392578
Successfully updated entry to database
0.005001544952392578
Successfully updated entry to database
0.006001472473144531
Successfully updated entry to database
0.006001472473144531
Successfully updated entry to database
0.007001638412475586
Successfully updated entry to database
0.007001638412475586
Successfully updated entry to database
0.007001638412475586
Successfully updated entry to database
0.008000373840332031
Successfully updated entry to database
0.008000373840332031
Successfully updated entry to database
0.009001016616821289
Successfully updated entry to database
0.009001016616821289
Successfully updat

In [27]:
# Update (精簡版)
columns = df.columns.tolist()
param_placeholders = ','.join([f'{col}=%s' for col in columns if col != 'id'])

def update(row):
    try:
        updateStatement = f"UPDATE profile SET {param_placeholders} WHERE id = %s"
        data = tuple(row[col] for col in columns if col != 'id') + (row['id'],)
        cur.execute(updateStatement, data)
        connection.commit()
        print(time.time()-t0)
        print("Successfully updated entry to database")
    except Error as e:
        print(f"Error updating entry to database: {e}")

t0 = time.time()
for index, row in df.iterrows():
    update(row)

0.0019998550415039062
Successfully updated entry to database
0.002998828887939453
Successfully updated entry to database
0.003998279571533203
Successfully updated entry to database
0.003998279571533203
Successfully updated entry to database
0.004997730255126953
Successfully updated entry to database
0.004997730255126953
Successfully updated entry to database
0.005999326705932617
Successfully updated entry to database
0.005999326705932617
Successfully updated entry to database
0.00699925422668457
Successfully updated entry to database
0.00699925422668457
Successfully updated entry to database
0.007998943328857422
Successfully updated entry to database
0.007998943328857422
Successfully updated entry to database
0.008998394012451172
Successfully updated entry to database
0.008998394012451172
Successfully updated entry to database
0.008998394012451172
Successfully updated entry to database
0.009998321533203125
Successfully updated entry to database
0.009998321533203125
Successfully updated

In [37]:
if connection.is_connected():
    # close the database connection
    connection.close()
    print("資料庫連線已關閉。")
else:
    print("資料庫連線未開啟。")

資料庫連線已關閉。
